In [1]:
from src.functions import *

# Global setting
with open(projectdir+'setting.json', 'r') as f:
    setting = json.load(f)
    _T = setting["T"]
    _t = setting["t"]
    _num_random_models = setting["num_random_models"]

# Load the JSON file
with open(projectdir+'instances.json', 'r') as f:
    instances = json.load(f)

with open(projectdir+'optimal_gammas.json', 'r') as f:
    optimal_gammas = json.load(f)

In [2]:
n_arr = [3, 4, 5, 6, 7, 8, 9]

time_lims = (0,50) # Range of time values
time_steps = 1000
time_range = np.linspace(time_lims[0], time_lims[1], time_steps)

In [3]:
optimal_gammas

{'3': 0.58947,
 '4': 0.51579,
 '5': 0.47368,
 '6': 0.43158,
 '7': 0.43158,
 '8': 0.42105,
 '9': 0.42105}

In [4]:
# NOTE: We evaluate the algorithm on the SAME 100 random instances for each point

In [5]:
def f(t,n):
    '''
    Calculate spectral gaps for quantum proposal.
    f(t) is defined to parallelize the time-loop.
    '''    
    delta_quantum_arr = np.zeros(_num_random_models)
    for i, instance in enumerate(instances[f'{n}']):
        m = IsingModel.from_coefficients(n, instance)
        # Calculate delta for quantum proposal
        proposal_mat_quantum = get_proposal_mat_quantum(m, optimal_gammas[f'{n}'], t)
        P_quantum = get_transition_matrix(m, _T, proposal_mat_quantum)
        delta_quantum = get_delta(P_quantum)
        delta_quantum_arr[i] = delta_quantum
    return delta_quantum_arr


In [6]:
delta_quantum_arr = np.zeros((len(n_arr), time_steps, _num_random_models))
for n_id, n in tqdm(enumerate(n_arr)):
    delta_quantum_arr[n_id] = np.asarray(joblib.Parallel(n_jobs=-1)(joblib.delayed(f)(t,n) for t in time_range))

0it [00:00, ?it/s]

1it [00:47, 47.02s/it]

2it [01:10, 32.98s/it]

3it [01:34, 28.91s/it]

4it [02:26, 38.01s/it]

5it [03:47, 53.79s/it]

6it [10:54, 180.60s/it]

7it [1:10:18, 1286.72s/it]

7it [1:10:18, 602.68s/it] 

# Save the data

In [7]:
filename = f'data/delta_vs_t.json'

grid_search_n = {
    'delta': delta_quantum_arr.tolist(),
    'qubits_range': n_arr,
    'num_random_models': _num_random_models,
    'time_range': time_range.tolist(),
    'T': _T,
    'gammas': optimal_gammas
}

save_in_json(grid_search_n, filename)